In [1]:
%%time
import findspark 
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
spark = SparkSession.builder.appName('i_s3_postgres').getOrCreate()

"""
bucket_name = "aatempjson2"
marca = 'elcomercio'
date_format = '20190301'
campaings = ['mundialista', 'semanasanta']

for campain in campaings:
    path_json_spark = 's3n://{}/{}/{}/{}/*.json'.format(bucket_name, marca, date_format, campain)
    print(path_json_spark)
    df_csv = spark.read.json(path_json_spark)
    df_csv.show()
    break
"""

CPU times: user 469 ms, sys: 256 ms, total: 725 ms
Wall time: 14.9 s


In [2]:
%%time
import json
import ast
from pyspark.sql import Row
# from pyspark.sql.types import StructType, StructField
# from pyspark.sql.types import DoubleType, IntegerType, StringType

bucket_name = "aatempjson2"
hour = 19
path_json_spark = 's3n://{}/{}/{}/{}/{}/*'.format(
    bucket_name, '2019', '04', '08', hour
)
text_parse = spark.read.text(
    path_json_spark
).collect()[0].\
    asDict()['value'].\
    replace('}{', '} {').\
    split(' ')

df_csv = spark.createDataFrame(
    [Row(**ast.literal_eval(_)) for _ in text_parse]
)
del text_parse

CPU times: user 10.8 s, sys: 182 ms, total: 10.9 s
Wall time: 1min 4s


In [3]:
%%time
df_csv.count()

CPU times: user 0 ns, sys: 3.34 ms, total: 3.34 ms
Wall time: 1.51 s


139680

In [4]:
df_csv.show()

+-------+--------+-----+-------+--------------------+
|campain|    date|event|project|                 url|
+-------+--------+-----+-------+--------------------+
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_dev|http://gitlab.ec....|
|  xxxxx|20190405|  ctr|ojo_

In [5]:
%%time
df_csv.groupby(['project', 'campain', 'date', 'event']).agg({'event': 'count'}).show()

+-------+-------+--------+-----+------------+
|project|campain|    date|event|count(event)|
+-------+-------+--------+-----+------------+
|ojo_dev|  xxxxx|20190405|  ctr|      139680|
+-------+-------+--------+-----+------------+

CPU times: user 19 ms, sys: 3.64 ms, total: 22.6 ms
Wall time: 1.96 s


In [28]:
print(dir(grr))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_df', '_jgd', 'agg', 'apply', 'avg', 'count', 'max', 'mean', 'min', 'pivot', 'sql_ctx', 'sum']


In [20]:
%%time
events = [_.event for _ in df_csv.select('event').distinct().collect()]

CPU times: user 24.2 ms, sys: 4.1 ms, total: 28.3 ms
Wall time: 2.14 s


In [21]:
events

['ctr']

In [23]:
%%time
from pyspark.sql.functions import col, asc

dicc_save = {}
for event in events:
    counter = df_csv.where(col("event") == event).count()
    #counter = df_csv.sql(s"SELECT col1 from table where col2>500 limit $q25)
    dicc_save[event] = counter
    print(event, counter)

received 500
ctr 500
CPU times: user 43.4 ms, sys: 11.2 ms, total: 54.7 ms
Wall time: 4min 27s


In [37]:
from pyspark.sql import Row
from collections import OrderedDict

def convert_to_row(d):
    return Row(
        **OrderedDict(sorted(d.items()))
    )

data = sc.parallelize(
    [dicc_save]
).map(convert_to_row).toDF()
data.show()

+---+--------+
|ctr|received|
+---+--------+
|500|     500|
+---+--------+



In [36]:

bucket_output = 'aaoutputattempjson2'
hoy = str(datetime.now()).replace(' ', '-').replace(':', '-').split('.')[0]
path = '{}.csv'.format(hoy)
path

'2019-04-05-14-29-51.csv'

In [40]:
data.write.mode('append').csv('s3n://{}/{}'.format(bucket_output, hoy))

In [3]:
ddff_validate = spark.read.csv('s3n://notification.output.test/**/*.csv')

ddff_validate.show()

+---+---+
|_c0|_c1|
+---+---+
|500|500|
|500|500|
+---+---+

